In [1]:
import pandas as pd

In [2]:
Dataset=pd.read_csv('CKD.csv')

In [3]:
Dataset.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3,0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2,0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1,0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1,0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0,0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [4]:
Dataset=pd.get_dummies(Dataset,dtype=int,drop_first=True)

In [5]:
Dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [6]:
Independent=Dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
Dependent=Dataset[['classification_yes']]

In [7]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(Independent,Dependent,test_size=0.30,random_state=0)

In [23]:
from sklearn.svm import SVC

from sklearn.model_selection import GridSearchCV

Param_grid={'kernel':['linear', 'rbf', 'poly', 'sigmoid'],
            'gamma': ['auto','scale'],
            'C':[10,100,1000,2000,3000]
            }
grid = GridSearchCV(SVC(probability=True), Param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 

grid.fit(X_train, Y_train)

Fitting 5 folds for each of 40 candidates, totalling 200 fits


c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [24]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

In [26]:
re=grid.cv_results_

In [27]:
grid_Pred=grid.predict(X_test)

In [29]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(grid_Pred,Y_test)
print(cm)

[[44  1]
 [ 1 74]]


In [30]:
from sklearn.metrics import classification_report
clf_report=classification_report(grid_Pred,Y_test)
print(clf_report)

              precision    recall  f1-score   support

           0       0.98      0.98      0.98        45
           1       0.99      0.99      0.99        75

    accuracy                           0.98       120
   macro avg       0.98      0.98      0.98       120
weighted avg       0.98      0.98      0.98       120



In [36]:
from sklearn.metrics import f1_score
f1_macro=f1_score(grid_Pred,Y_test)
print("The f1 sore of the best parameter{}:".format(grid.best_params_),f1_score)
print(f1_macro)

The f1 sore of the best parameter{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}: <function f1_score at 0x00000264D07A0720>
0.9866666666666667


In [34]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Y_test,grid.predict_proba(X_test)[:,1])

0.9997037037037036

In [35]:
table=pd.DataFrame.from_dict(re)
print(table)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_C  \
0        0.013342      0.000813         0.011933        0.001231      10   
1        0.013119      0.004931         0.012092        0.001050      10   
2        0.015158      0.001003         0.010487        0.001997      10   
3        0.010180      0.001597         0.015745        0.007813      10   
4        0.007190      0.000742         0.012162        0.000977      10   
5        0.012512      0.003867         0.016197        0.009131      10   
6        0.015455      0.000684         0.010773        0.001532      10   
7        0.009775      0.001282         0.011874        0.003130      10   
8        0.009724      0.002924         0.016735        0.012415     100   
9        0.010191      0.003009         0.011288        0.002731     100   
10       0.016062      0.001358         0.010877        0.001285     100   
11       0.008459      0.001916         0.014819        0.007014     100   
12       0.0